In [80]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import os

# Load the MNIST dataset

In [81]:
#loading the dataset
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize the data

In [82]:
X_train = (X_train.astype(np.float32) -127.5)/127.5
X_train = np.expand_dims(X_train, axis = -1)

# Shuffling the images

In [83]:
BUFFER_SIZE = 60000
BATCH_SIZE = 128

dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Building Generator and Discriminator

In [84]:
#building generator
def build_generator():
  model = tf.keras.Sequential([
    Dense(256, input_shape=(100,)),
    LeakyReLU(0.2),
    Dense(512),
    LeakyReLU(0.2),
    Dense(1024),
    LeakyReLU(0.2),
    Dense(784, activation='tanh'),
    Reshape((28, 28, 1))
  ])
  return model

#building discriminator
def build_discriminator():
  model = tf.keras.Sequential([
    Dense(512, input_shape=(28, 28, 1)),
    LeakyReLU(0.2),
    Dense(256),
    LeakyReLU(0.2),
    Dense(1, activation='sigmoid')
  ])
  return model

# Initialize the models

In [85]:
generator = build_generator()
discriminator = build_discriminator()

# Loss function and optimization

In [86]:
#setting up the loss function
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()

#optimization
gen_optimization = tf.keras.optimizers.Adam(1e-4)
disc_optimization = tf.keras.optimizers.Adam(1e-4)

# Training GANs

In [87]:
def train_step(images):
  noise = tf.random.normal([BATCH_SIZE, 100])

  #---Train Discriminator---
  with tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training = True)
    
    real_output = discriminator(images, training = True)
    fake_output = discriminator(generated_images, training = True)

    real_loss = cross_entropy_loss(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy_loss(tf.zeros_like(fake_output), fake_output)

    disc_loss = real_loss + fake_loss

  disc_grad = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  disc_optimization.apply_gradients(zip(disc_grad, discriminator.trainable_variables))


  #---Train Generator---
  with tf.GradientTape() as gen_tape:
    generated_images = generator(noise, training = True)
    fake_output = discriminator(generated_images, training = True)
    gen_loss = cross_entropy_loss(tf.ones_like(fake_output), fake_output)
  
  gen_grad = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gen_optimization.apply_gradients(zip(gen_grad, generator.trainable_variables))

  return gen_loss, disc_loss

# Saving generated images

In [88]:
#Track losses
gen_losses = []
disc_losses = []

#make a dir to save generated images
os.makedirs("Generated_Images", exist_ok = True)

#Generate images and save them
def gen_save_images(model, epoch, test_input):
  predictions = model(test_input, training = False)
  fig = plt.figure(figsize=(5, 5))
  for i in range(predictions.shape[0]):
    plt.subplot(5, 5, i+1)
    plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.axis('off')
  plt.suptitle(f"Epoch {epoch}")
  plt.tight_layout()
  plt.savefig(f"Generated_Images/image_epoch_{epoch:03d}.png")
  plt.show()
  plt.close()

# Training Model

In [89]:
def train_model(dataset, epochs=50):
  seed = tf.random.normal([25, 100])

  for epoch in range(epochs):
    for image_batch in dataset:
      gen_loss, disc_loss = train_step(image_batch)

    #Track loss
    gen_losses.append(gen_loss.numpy())
    disc_losses.append(disc_loss.numpy())

    #Print epochs
    print(f'Epoch: {epoch+1}, Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')
    gen_save_images(generator, epoch+1, seed)  

In [ ]:
train_model(dataset, epochs = 100)5

# Plotting the losses 

In [ ]:
  plt.figure(figsize=(10, 5))
  plt.plot(gen_losses, label='Generator Loss')
  plt.plot(disc_losses, label='Discriminator Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Trainin Losses')
  plt.legend
  plt.grid(True)
  plt.show()

# Display Generated Images

In [76]:
def disp_generated_images():
  noise = tf.random.normal([25, 100])
  generated_images = generator(noise, training = False)

  fig = plt.figure(figsize=(5, 5))
  for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.axis('off')

  plt.suptitle("Final Generated Images")
  plt.tight_layout()
  plt.show()

In [ ]:
disp_generated_images()

# Performance Evaluation

In [78]:
def eval_disc():
  noise = tf.random.normal([BATCH_SIZE, 100])
  generated_image = generator(noise, training = False)
  real_image = next(iter(dataset))

  real_output = discriminator(real_image, training = False)
  fake_output = discriminator(generated_image, training = False)

  real_acc = tf.reduce_mean(tf.cast(real_output > 0.5, tf.float32))
  fake_acc = tf.reduce_mean(tf.cast(fake_output < 0.5, tf.float32))

  print(f'Real Accuracy: {real_acc:.4f}, Fake Accuracy: {fake_acc:.4f}')

In [ ]:
eval_disc()